<a href="https://colab.research.google.com/github/isaachcamp/geofm-plant-traits/blob/main/test_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import packages
from pathlib import Path
import json

import pandas as pd
import ee
import geemap

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Authenticate google credentials and initialise the GEE project.
ee.Authenticate()
ee.Initialize(project='gsfm-plant-traits')

In [ ]:
!git clone https://github.com/isaachcamp/geofm-plant-traits.git

In [ ]:
dpath = Path('/content/drive/MyDrive/IECDT/PhD/chile_data')

In [ ]:
sites = pd.read_excel(dpath / 'ARBOLES-Chile_data_Soil.xlsx', sheet_name='Sites Metadata')
census = pd.read_excel(dpath / 'ARBOLES-Chile_data_Soil.xlsx', sheet_name='Census')
nutrients = pd.read_excel(dpath / 'ARBOLES-Chile_data_Soil.xlsx', sheet_name='Leaf nutrients')

In [ ]:
# Identify valid geolocated plots only (must have at least four vertices).

# Get vertices of each plot and drop plots with NaNs.
plot_vertices = sites[['X_dec', 'Y_dec', 'Plot_ID', 'Vertex']]
plot_vertices.dropna(inplace=True)

# Group by Plot_ID and check if each group has exactly four entries (vertices)
vertex_counts = plot_vertices.groupby('Plot_ID')['Plot_ID'].count()
geolocated_ids = vertex_counts[vertex_counts == 4].index.tolist()

# Filter the DataFrame to keep only rows of Plot_IDs with four vertices
geolocated_plots = sites[sites['Plot_ID'].isin(geolocated_ids)]
geolocated_plots.reset_index(drop=True, inplace=True)

print(f'Number of geolocated plots: {geolocated_plots["Plot_ID"].unique().size}')

Number of geolocated plots: 9


In [ ]:
# Mask out plots without all the required data.
required_cols = ['Leaf traits (Morphological and nutrients)', 'Census']
valid_plots_mask = ((geolocated_plots[required_cols[0]] == 'Y')
                      & (geolocated_plots[required_cols[1]] == 'Y'))
valid_plots = geolocated_plots[valid_plots_mask]

print(f'Number of valid geolocated plots: {valid_plots["Plot_ID"].unique().size}')

Number of valid geolocated plots: 6


In [ ]:
# Create a GeoJSON file with plot geometries.

# Create a list to store Features for each plot_ID
features = []

for plot_id in valid_plots['Plot_ID'].unique():
  # Filter the valid_plots DataFrame for the current plot_ID
  plot_coords = valid_plots[valid_plots['Plot_ID'] == plot_id][['X_dec', 'Y_dec']].values.tolist()
  plot_coords = [[*plot_coords, plot_coords[0]]]

  feature = {
      "type": "Feature",
      "properties": {
          "Plot_ID": plot_id,
      },
      "geometry": {
          "type": "Polygon",  # Assuming your data has point geometries
          "coordinates": plot_coords
      }
  }
  features.append(feature)

# Create the GeoJSON object
geojson_plots = {
  "type": "FeatureCollection",
  "features": features
}

# Save the GeoJSON data to a file
if not (dpath / 'valid_plots.geojson').exists():
  with open(dpath / 'valid_plots.geojson', 'w') as f:
    json.dump(geojson_plots, f)

In [ ]:
geojson_plots = json.load(open(dpath / 'valid_plots.geojson'))

In [ ]:
# Create a collection of country borders using GEE's available feature collections.
# Then filter Features to remove all country borders except Chile.
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
chile = countries.filter(ee.Filter.eq('country_na', 'Chile'))
plots = ee.FeatureCollection(geojson_plots)

In [ ]:
# Create a Map object for visualisation, and add Chile vis layer.
map = geemap.Map()
map.addLayer(chile, {}, 'Chile')
map.addLayer(plots, {}, 'Plots')
map.centerObject(plots.first(), 16)
map

Map(center=[-34.21263977177649, -71.19334794098509], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# Get intersection of plots with Sentinel-2 level 2a data (10m resolution)

# Required bands.
required_bands = ['B8', 'B4', 'B3', 'B2']

# Define the date range for Sentinel-2 imagery
start_date = '2020-06-01'
end_date = '2020-7-31'

# Define the cloud cover threshold
cloud_cover_max = 5

# Load Sentinel-2 Level 2A collection
s2_plots = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterDate(start_date, end_date)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover_max))
      .filterBounds(plots)
      .select(required_bands)
)

s2_plots

In [ ]:
# Function to clip and calculate NDVI for each image
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi).clip(plots)

# Map the function to the Sentinel-2 collection
s2_ndvi = s2.map(calculate_ndvi)

# Reduce the collection to get the median NDVI for the time period
median_ndvi = s2_ndvi.select('NDVI').median()

# Visualize the median NDVI
vis_params = {'min': 0, 'max': 0.8, 'palette': ['red', 'yellow', 'green']}
map.addLayer(median_ndvi, vis_params, 'Median NDVI')

# Export the median NDVI to Google Drive
# task = ee.batch.Export.image.toDrive(**{
#     'image': median_ndvi,
#     'description': 'Median_NDVI_Chile_Plots',
#     'folder': 'GEE_Exports',
#     'fileNamePrefix': 'Median_NDVI_Chile_Plots',
#     'region': plots.geometry(),
#     'scale': 10,
#     'maxPixels': 1e13
# })

# task.start()

# print('Exporting Median NDVI to Google Drive...')


In [ ]:
census['Plot_ID'] = census['Site'] + '-0' + census['Plot'].astype(str)

In [ ]:
census[census['Plot_ID'].isin(valid_plots['Plot_ID'].unique())]

,Measurement,ID_Site,Site,Plot,ID_Tree,Especie,DBH (cm),AB (m2),Tree health,Dieback,Observaction,Plot_ID
0,2019-11-05,1,CAB,1,1,Pb,10.5,0.008659,NaN,NaN,dBase 2019 of Sergio Donoso,CAB-01
1,2019-11-05,1,CAB,1,8,Pb,12.0,0.011310,NaN,NaN,dBase 2019 of Sergio Donoso,CAB-01
2,2019-11-05,1,CAB,1,8,Pb,13.0,0.013273,NaN,NaN,dBase 2019 of Sergio Donoso,CAB-01
3,2019-11-05,1,CAB,1,8,Pb,12.0,0.011310,NaN,NaN,dBase 2019 of Sergio Donoso,CAB-01
4,2019-11-05,1,CAB,1,8,Pb,11.0,0.009503,NaN,NaN,dBase 2019 of Sergio Donoso,CAB-01
...,...,...,...,...,...,...,...,...,...,...,...,...
1666,2020-02-05,3,SPT,3,378,Lp,69.8,0.382649,NaN,NaN,NaN,SPT-03
1667,2020-02-05,3,SPT,3,379,Sc,61.3,0.295128,NaN,NaN,NaN,SPT-03
1668,2020-02-05,3,SPT,3,380,Lp,17.3,0.023506,NaN,NaN,NaN,SPT-03
1669,2020-02-05,3,SPT,3,381,Lp,12.1,0.011499,NaN,NaN,NaN,SPT-03


In [ ]:
nutrients

,Site Number,Site,ARCHIVO,ID LAB,Species,ID SAMPLE,N (%),P (%),N/P (%),K (%),Ca (%),Mg (%)
0,1,CAB,vURRU10-21,363,Lc,16,1.01,0.07,14.4,0.86,0.49,0.11
1,1,CAB,vURRU10-21,364,Lc,101,0.75,0.04,18.8,0.56,0.63,0.15
2,1,CAB,vURRU10-21,365,Lc,157,0.92,0.05,18.4,0.77,1.05,0.24
3,1,CAB,vURRU10-21,366,Lc,429,0.69,0.08,8.6,0.49,1.10,0.23
4,1,CAB,vURRU10-21,367,Lc,483,0.90,0.05,18.0,0.75,0.91,0.16
...,...,...,...,...,...,...,...,...,...,...,...,...
80,5,COR,vURRU05-21,318,Lp,4,1.12,0.06,18.7,0.65,1.26,0.44
81,5,COR,vURRU05-21,319,Lp,5,1.20,0.09,13.3,1.50,1.28,0.53
82,5,COR,vURRU05-21,320,Lp,6,1.41,0.08,17.6,1.02,1.51,0.54
83,5,COR,vURRU05-21,321,Lp,7,1.28,0.07,18.3,0.98,1.18,0.32
